In [ ]:
from astropy.stats import sigma_clipped_stats
from photutils import aperture_photometry
from photutils import CircularAperture, CircularAnnulus, RectangularAperture
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import numpy as np


imglist = open("imageList.txt") #файл imageList.txt содержит список имён всех кадров bcd. Каждое имя с новой строки
fullphot = Table()
for bcdname in imglist:
    bcdname = bcdname[:-1]
    errname = bcdname.replace("bcd.fits", "bunc.fits")
    maskname = bcdname.replace("bcd.fits", "bimsk.fits")
    img_hdu = fits.open(bcdname)
    err_hdu = fits.open(errname)
        
    head = img_hdu[0].header
    date = head["BMJD_OBS"]
    #Определение координат объекта
    head['NAXIS']=2
    del head['NAXIS3']
    w=WCS(img_hdu[0].header)
    center = w.all_world2pix(330.7946776,18.8841286, 0) #координаты объекта (обновлять вручную)
    meanimage = np.mean(img_hdu[0].data, axis=0)
    meanerror = np.mean(err_hdu[0].data, axis=0) 
    
                                 
    badpix=((6,31,26,23,26,26),(24,17,13,14,1,13))  #Следующие 6 строк - плохие пиксели (обновлять вручную, порядок координат обратный относительно фитса)
    mask = np.zeros(meanimage.shape, dtype=bool)
    mask[badpix] = True
    mask2 = np.zeros(meanimage.shape, dtype=bool)
    mask2[0:32,0:32] = True
    mask2[badpix] = False
    
    #Апертурная фотометрия
    data = meanimage
    aperture = CircularAperture(center, r=4)
    annulus_aperture = CircularAnnulus(center, r_in=12, r_out=24)
    
    annulus_masks = annulus_aperture.to_mask(method='center')
    
    bkg_median = []
    
    error = meanerror
    
    annulus_data = annulus_masks.multiply(data)
    
    tmp_data = np.ones((32,32))
    tmp_data=mask2*tmp_data
    tmp_annulus_data = annulus_masks.multiply(tmp_data)
    (tmp_annulus_data > 0)
    annulus_data_1d = annulus_data[tmp_annulus_data > 0]
    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
    bkg_median.append(median_sigclip)
    bkg_median = np.array(bkg_median)
    phot = aperture_photometry(data, aperture, error=error)
    phot['MJD'] = date
    phot['annulus_median'] = bkg_median
    phot['aper_bkg'] = bkg_median * aperture.area
    phot['aper_sum_bkgsub'] = phot['aperture_sum'] - phot['aper_bkg']
    phot=Table(phot)
    fullphot = vstack([fullphot, phot])
fullphot.write("predresult.txt", format='ascii')